In [ ]:
# https://www.kaggle.com/competitions/cassava-leaf-disease-classification/discussion/200308

In [13]:
import os                                         # To work with OS commands
import pandas as pd
from PIL import Image                             # To read Images
from termcolor import colored                     # To Colorfull output
from datetime import datetime                     # To calculate durations

import numpy as np                                # To work with Numpy arrays
import matplotlib.pyplot as plt                   # To Visualization
import seaborn as sns                             # To Visualization

import torch                                      # To work with TORCH framework
import torch.nn as nn                             # To work with Neural Networks
import torchvision                                # To work with image datasets
import torchvision.transforms as transforms       # To create data transforms
from torch.utils.data import Dataset, DataLoader,random_split

from torchvision.models import efficientnet_b5, EfficientNet_B5_Weights # Pretrained model with its weights

from sklearn.metrics import confusion_matrix, classification_report # To Evaluate the result

### JUST Write your Model Structure here and then import model which will fix up your weights and biases . We have confirmed this!
By Checking the model weights . Also when running  'if param.requires_grad:' , We get the convol layer[6] too which We define in
our custom model but didnt write in this notebook . but still we get that !! So it means Our model is
fully loaded by this steps only !!

In [20]:
from torchvision.models import vgg16
vgg16_model= vgg16

num_classes =5

class CNN(nn.Module) :
    def __init__(self, num_classes) :
        super(CNN, self).__init__()
        ####  CONVs
        self.conv_layers = vgg16_model     
        ####  Dense
        self.dense_layers = nn.Sequential(
            nn.Linear(1000, 256),  # UPDATE !efficientnet_model has 1000 classes in its layer thats why we take from there
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 5)    # The number 5 corresponds to the number of output classes
        )
    def forward(self, X) :
        out = self.conv_layers(X)
         # the output tensor is typically multi-dimensional (e.g., [batch_size, channels, height, width]). out.size(0): This is the batch size
        out = out.view(out.size(0), -1)    # used to flatten the output tensor
        out = self.dense_layers(out)
        return out

### Dont miss this bcz We saved our model in CPU model in the actual notebook 

> map_location=torch.device('cpu') 

In [22]:
# Load the model with map_location set to 'cpu'
model = torch.load('/kaggle/input/my-full-model/my_VGG16_full_model.pth', map_location=torch.device('cpu'))

print(type(model))  # This should now show the correct type
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

model.to(device)

<class '__main__.CNN'>
cpu


CNN(
  (conv_layers): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16)

In [23]:
# Print weights after training
for name, param in model.named_parameters():
    if param.requires_grad:                     
        ## we get conv-layers[6] printed here bcz We unfreeze the last layer as 'true' so that it get trained..that means our model was correctly loaded
        print(f"Layer: {name}, Weight shape: {param.data.shape}")
        print(f"Trained weights:\n{param.data}\n")


Layer: conv_layers.classifier.6.weight, Weight shape: torch.Size([1000, 4096])
Trained weights:
tensor([[ 0.0391,  0.0279, -0.0045,  ..., -0.0164, -0.0181, -0.0192],
        [ 0.0311,  0.0255, -0.0510,  ...,  0.0260, -0.0188, -0.0106],
        [-0.0236,  0.0164,  0.0128,  ..., -0.0068,  0.0855, -0.0327],
        ...,
        [ 0.0167,  0.0009,  0.0049,  ...,  0.0102, -0.0417, -0.0250],
        [ 0.0436,  0.0110, -0.0246,  ..., -0.0044,  0.0169, -0.0395],
        [ 0.0191, -0.0257, -0.0060,  ...,  0.0687, -0.0122,  0.0033]])

Layer: conv_layers.classifier.6.bias, Weight shape: torch.Size([1000])
Trained weights:
tensor([ 0.0179, -0.0360, -0.0027,  0.0054,  0.0230,  0.0027,  0.0064, -0.0129,
        -0.0144,  0.0109, -0.0150, -0.0120,  0.0153,  0.0169,  0.0120, -0.0238,
        -0.0029, -0.0006,  0.0176,  0.0165,  0.0051,  0.0268, -0.0049,  0.0199,
        -0.0108, -0.0192, -0.0255, -0.0315, -0.0052,  0.0288,  0.0063, -0.0130,
        -0.0231, -0.0064,  0.0057, -0.0118, -0.0053,  0.0142,

In [ ]:
transform = transforms.Compose(
    [        
        transforms.ToTensor(),                         # Convert images to tensor
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize R, G, B channels
    ]
)

BATCH_SIZE = 32

In [ ]:
test_dir='/kaggle/input/cassava-leaf-disease-classification/test_images'

class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_names = [f for f in os.listdir(root_dir) if os.path.isfile(os.path.join(root_dir, f))]
    
    def __len__(self):
        return len(self.image_names)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_names[idx])  # Get image path
        image = Image.open(img_name).convert('RGB')  # Open image and convert to RGB
        
        if self.transform:
            image = self.transform(image)  # Apply transformations if any
        
        return image, self.image_names[idx]  # Return image and filename
    
print("transforming test images to tensors done!!!!!!!")


test_dataset = TestDataset(root_dir=test_dir, transform=transform)
                           
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

num_files = len(test_dataset)

print(f'Number of files in the test dataset: {num_files}')




In [ ]:
# Ensure the model is in evaluation mode
#model.to(device)
model.eval()

# Initialize lists to store predictions and filenames
predictions = []
img_names = []

# Load the dataset
test_dataset = TestDataset(root_dir=test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Start prediction
t0 = datetime.now()

with torch.no_grad():
    for images, file_names in test_loader:
        images = images.to(device)                 ## ENSURE IT !!!!!!!!!
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        
        # Convert predictions and filenames to lists
        predictions.extend(preds.cpu().numpy())
        img_names.extend(file_names)

# Create a DataFrame for submission
submission_df = pd.DataFrame({
    'image_id': img_names,
    'label': predictions
})

# Save the DataFrame to a CSV file
submission_df.to_csv('/kaggle/working/submission.csv', index=False)

# Calculate and print duration
dt = datetime.now() - t0
print(f"Prediction Duration: {dt}")

print("Submission CSV created successfully.")
